## Interfaces: from protocols to ABCs


In [1]:
# 看看　sequnce protocol 的强大

class Foo:
    
    def __getitem__(self, pos):
        return range(0, 30, 10)[pos]
    
    

In [2]:
f = Foo()
f[1]

10

In [3]:
print(f[:])
print(f[1:])

range(0, 30, 10)
range(10, 30, 10)


In [4]:
for i in f:
    print(i)

0
10
20


In [5]:
10 in f

True

In [6]:
15 in f

False

仅仅实现了一个 getitem 方法，就可以支持切片，支持迭代，支持判断

In [32]:
# 简单验证 __iter__ 的作用

class Z:
    
    def __init__(self, seq):
        self.seq = list(seq)
        
    def __iter__(self):
        
        return iter(self.seq)
    
    def __str__(self):
        
        return str(tuple(self))
    
    def __eq__(self, other):
        
        return tuple(self) == tuple(other)
    
class ZZ:
    
    def __init__(self, seq):
        self.seq = list(seq)
        
    def __eq__(self, other):
        
        return tuple(self) == tuple(other)


In [24]:
z = Z(range(10))

for i in z:
    print(i)

0
1
2
3
4
5
6
7
8
9


In [25]:
print(z)

(0, 1, 2, 3, 4, 5, 6, 7, 8, 9)


In [26]:
z1 = Z(range(5))
z2 = Z(range(5))

z1 == z2

True

In [27]:
z1

In [28]:
tuple(z1)

(0, 1, 2, 3, 4)

In [33]:
zz = ZZ(range(5))

for i in zz:
    print(i)

TypeError: 'ZZ' object is not iterable

In [34]:
print(zz)

In [35]:
z1 = ZZ(range(5))
z2 = ZZ(range(5))

z1 == z2

TypeError: 'ZZ' object is not iterable

从以上的代码可以看到

- iter 方法可将 cls 变为可迭代的对象
- 在 eq, str 方法中可以依托于 iter 来写

### setitem 与不可变 

In [36]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchMark:
    
    ranks = [str(i) for i in range(2, 11)] + list('AJQK')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        
        self.cards = [Card(rank, suit) for rank in self.ranks
                                       for suit in self.suits]
        
    def __len__(self):
        
        return len(self.cards)
    
    def __getitem__(self, pos):
        
        return self.cards[pos]


In [37]:
# 这是 chapter1 中的例子
# 现在对生成的牌做 打乱 处理

import random

f = FrenchMark()
random.shuffle(f)

print(f)

TypeError: 'FrenchMark' object does not support item assignment

In [39]:
#  'FrenchMark' object does not support item assignment
#  报错是因为 f 仅实现了不可变序列协议
# 要使 f 支持可变协议，　添加 __setitem__ 方法即可

def set_cards(ins, pos, val):
    
    ins.cards[pos] = val
    
FrenchMark.__setitem__ = set_cards

f2 = FrenchMark()
random.shuffle(f2)

print(f2)
print(f2[:5])

[Card(rank='K', suit='hearts'), Card(rank='K', suit='clubs'), Card(rank='Q', suit='spades'), Card(rank='2', suit='hearts'), Card(rank='8', suit='clubs')]


注意：

- self.cards 为 list，是可变的，但这与 f/f2 无关
- shuffle 操作针对的是 f/f2，　而不是 self.cards

In [41]:
# 无处不在的 collections.abc

class ZX:
    
    def __len__(self):
        return len(range(5))
    
from collections import abc
isinstance(ZX(), abc.Sized)

True

### Subclassing an ABC


In [ ]:
import collections

Card = collections.namedtuple('Card', 'rank suit')

class FrenchDeck2(collections.MutableSequence):
    
    ranks = [str(i) for i in range(2, 11)] + list('JQKA')
    suits =  'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]
        
    def __len__(self):
        
        return len(self._cards)
    
    def __getitem__(self, pos):
        return self._cards[pos]
    
    def __setitem__(self, pos, val):
        
        self._cards[pos] = val
        
    def __delitem__(self, pos):
        del self._cards[pos]
        
    def insert(self, pos, val):
        
        self._cards.insert(pos, val)
        
    
        

### ABCs in the standard library.


一张结构清晰的图片：

![](Selection_001.jpg)

In [42]:

import abc

class Tombola(abc.ABC):
    
    @abc.abstractmethod
    def load(self, iterable):
        '''add items from an iterable'''
        
    @abc.abstractmethod
    def pick(self):
        '''remove item at random, return it.
        this method should raise LookupError when instance is empty'''
        
    def loaded(self):
        '''return True if theres at least 1 item false otherwise '''
        return bool(self.inspect())
    
    def inspect(self):
        '''return a sorted tuple with the items currently inside'''
        items = []
        while True:
            try:
                items.append(self.pick())

            except LookupError:
                break
                
        self.load(items)
        return tuple(sorted(items))
    

In [43]:
class Fake(Tombola):
    
    def pick(self):
        return 123
    
    

In [44]:
Fake

__main__.Fake

In [45]:
f = Fake()

TypeError: Can't instantiate abstract class Fake with abstract methods load

### Subclassing the Tombola ABC


In [53]:
import random

class BingoCage(Tombola):
    
    def __init__(self, items):
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)
        
    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)
        
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
            
    def __call__(self):
        
        self.pick()


In [48]:
b = BingoCage(range(5))
b.pick()

0

In [50]:
b.pick()

1

In [51]:
b.load(range(10, 15))

In [52]:
b.pick()

12